In [29]:
import os
import pandas as pd
import numpy as np

In [30]:
# Load the dataset

# Path to the dataset
path = '../dataset/yolo_signals_cbba/raw_metadata.csv'

# Load the dataset
metadata_df = pd.read_csv(path)
metadata_df.head()

,timestamp,image_name,exif_image_width,exif_image_height,labels,latitude,latitude_ref,longitude,longitude_ref,altitude,altitude_ref,speed
0,2024-08-18 17:37:25,2024_08_18_17_37_25_601_-0400_1280x1280.left-r...,1280,1280,['zona-escolar'],-17.428987,S,-66.159564,W,2596.5,b'\x00',27.8943
1,2024-08-18 17:40:54,2024_08_18_17_40_54_602_-0400_1280x1280.left-r...,1280,1280,"['zona-escolar', 'prohibido-girar-izquierda']",-17.417096,S,-66.157226,W,2600.3,b'\x00',34.4967
2,2024-08-18 17:40:55,2024_08_18_17_40_55_571_-0400_1280x1280.left-r...,1280,1280,"['prohibido-girar-izquierda', 'zona-escolar']",-17.417017,S,-66.157214,W,2600.3,b'\x00',27.9167
3,2024-08-18 17:40:57,2024_08_18_17_40_57_532_-0400_1280x1280.left-r...,1280,1280,['prohibido-girar-izquierda'],-17.416900,S,-66.157191,W,2600.3,b'\x00',16.9381
4,2024-08-18 17:42:10,2024_08_18_17_42_10_543_-0400_1280x1280.right-...,1280,1280,['limite-velocidad-40'],-17.412288,S,-66.156361,W,2599.3,b'\x00',31.8807


In [31]:
# Check the shape of the dataset
metadata_df.shape

# check unique values in the image_name column
metadata_df['image_name'].nunique()


1167

In [32]:
# transform labels, remove square brackets, convert to string, remove leading and trailing whitespaces, remove whitespaces between commas
metadata_df['labels'] = metadata_df['labels'].apply(lambda x: x.replace('[', ''))
metadata_df['labels'] = metadata_df['labels'].apply(lambda x: x.replace(']', ''))
metadata_df['labels'] = metadata_df['labels'].apply(lambda x: x.replace('\'', ''))
metadata_df['labels'] = metadata_df['labels'].apply(lambda x: x.strip())
metadata_df['labels'] = metadata_df['labels'].apply(lambda x: x.replace(', ', ','))
metadata_df.head()

,timestamp,image_name,exif_image_width,exif_image_height,labels,latitude,latitude_ref,longitude,longitude_ref,altitude,altitude_ref,speed
0,2024-08-18 17:37:25,2024_08_18_17_37_25_601_-0400_1280x1280.left-r...,1280,1280,zona-escolar,-17.428987,S,-66.159564,W,2596.5,b'\x00',27.8943
1,2024-08-18 17:40:54,2024_08_18_17_40_54_602_-0400_1280x1280.left-r...,1280,1280,"zona-escolar,prohibido-girar-izquierda",-17.417096,S,-66.157226,W,2600.3,b'\x00',34.4967
2,2024-08-18 17:40:55,2024_08_18_17_40_55_571_-0400_1280x1280.left-r...,1280,1280,"prohibido-girar-izquierda,zona-escolar",-17.417017,S,-66.157214,W,2600.3,b'\x00',27.9167
3,2024-08-18 17:40:57,2024_08_18_17_40_57_532_-0400_1280x1280.left-r...,1280,1280,prohibido-girar-izquierda,-17.416900,S,-66.157191,W,2600.3,b'\x00',16.9381
4,2024-08-18 17:42:10,2024_08_18_17_42_10_543_-0400_1280x1280.right-...,1280,1280,limite-velocidad-40,-17.412288,S,-66.156361,W,2599.3,b'\x00',31.8807


In [33]:
# denormalize the labels column to another dataframe, split by comma and create a new row for each label, use the column image_name as index
labels_df = metadata_df['labels'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('labels')
labels_df.index = labels_df.index.map(metadata_df['image_name'])
# set index name
labels_df.index.name = 'image_name'
labels_df.head()

image_name
2024_08_18_17_37_25_601_-0400_1280x1280.left-region.jpg                 zona-escolar
2024_08_18_17_40_54_602_-0400_1280x1280.left-region.jpg                 zona-escolar
2024_08_18_17_40_54_602_-0400_1280x1280.left-region.jpg    prohibido-girar-izquierda
2024_08_18_17_40_55_571_-0400_1280x1280.left-region.jpg    prohibido-girar-izquierda
2024_08_18_17_40_55_571_-0400_1280x1280.left-region.jpg                 zona-escolar
Name: labels, dtype: object

In [34]:
# save labels_df to a csv file
path = '../dataset/yolo_signals_cbba/labels.csv'
labels_df.to_csv(path, index=True)

In [35]:
# Save the transformed dataset
path = '../dataset/yolo_signals_cbba/transformed_metadata.csv'
metadata_df.to_csv(path, index=False)

In [36]:
metadata_df.head()

,timestamp,image_name,exif_image_width,exif_image_height,labels,latitude,latitude_ref,longitude,longitude_ref,altitude,altitude_ref,speed
0,2024-08-18 17:37:25,2024_08_18_17_37_25_601_-0400_1280x1280.left-r...,1280,1280,zona-escolar,-17.428987,S,-66.159564,W,2596.5,b'\x00',27.8943
1,2024-08-18 17:40:54,2024_08_18_17_40_54_602_-0400_1280x1280.left-r...,1280,1280,"zona-escolar,prohibido-girar-izquierda",-17.417096,S,-66.157226,W,2600.3,b'\x00',34.4967
2,2024-08-18 17:40:55,2024_08_18_17_40_55_571_-0400_1280x1280.left-r...,1280,1280,"prohibido-girar-izquierda,zona-escolar",-17.417017,S,-66.157214,W,2600.3,b'\x00',27.9167
3,2024-08-18 17:40:57,2024_08_18_17_40_57_532_-0400_1280x1280.left-r...,1280,1280,prohibido-girar-izquierda,-17.416900,S,-66.157191,W,2600.3,b'\x00',16.9381
4,2024-08-18 17:42:10,2024_08_18_17_42_10_543_-0400_1280x1280.right-...,1280,1280,limite-velocidad-40,-17.412288,S,-66.156361,W,2599.3,b'\x00',31.8807
